In [2]:
import os
import numpy as np
import pandas as pd
import pickle
from scipy.optimize import least_squares
from scipy.special import kv

Load the results from HALS generated with "04_HALS_strain.ipynb" and "04_HALS_head.ipynb"

In [3]:
# Load results

path = 'Results/pkl_files/'

with open(path + 'head.pkl', 'rb') as f:
    head = pickle.load(f)
    
with open(path + 'strain.pkl', 'rb') as f:
    strain = pickle.load(f)
    
# Merge in one dic
strain['amp_strain'] = strain.pop('amp')
strain['phase_strain'] = strain.pop('phase')
head['amp_head'] = head.pop('amp')
head['phase_head'] = head.pop('phase')


data = {**head, **strain}

data.keys()

dict_keys(['complex', 'amp_head', 'phase_head', 'amp_strain', 'phase_strain'])

Compute phase amplitude ratio and phase shift

In [4]:
# Compute amplitude ratio
amp_ratio = data['amp_head'] / ((data['amp_strain'] * 1E-9))

# Compute phase shift
phase_shift = np.rad2deg(data['phase_strain'] - data['phase_head'])

Now we script a function to invert Wang, 2018 analytical solution

In [5]:
# Functions that are in Wang 2018 solution. We write it this way to keep it clean

def omega_fn(PERIOD):
    params = 2 * np.pi / PERIOD
    return params

def betta_fn(K_LE, K_AQ, B_AQ, B_LE, omega, S_AQ):
    params = ((K_LE / (K_AQ * B_AQ * B_LE)) + ((1j * omega * S_AQ * B_AQ) / (K_AQ * B_AQ))) ** 0.5
    return params

def argument_fn(omega, S_AQ, B_AQ, K_LE, B_LE):
    params = (1j * omega * S_AQ * B_AQ) / (1j * omega * S_AQ * B_AQ + K_LE / B_LE)
    return params

def tide_fn(S_AQ, E_0):
    params = (E_0) / (S_AQ)
    return params

def xi_fn(R_W, R_C, omega, K_AQ, B_AQ, betta):
    params = 1 + ((1j * omega * R_W) / (2 * K_AQ * B_AQ * betta)) * (kv(0, betta * R_W) / kv(1, betta * R_W)) * (R_C / R_W)**2
    return params

def h_w_fn(argument, tide, xi):
    params = argument * tide / xi
    return params

def drawdown_fn(omega, R_C, h_w, betta, K_AQ, B_AQ, R):
    params = -(1j * omega * R_C ** 2 * h_w * kv(0, betta * R)) / (2 * K_AQ * B_AQ * betta * R_W * kv(1, betta * R))
    return params

def flux_fn(omega, h_w,  R_C):
    params = omega * np.absolute(h_w) * np.pi * R_C**2 *1E3
    return params

# Wang 2018 solution
def wang_2018(K_AQ, S_AQ, K_LE):
    
    omega = omega_fn(PERIOD)
    betta = betta_fn(K_LE, K_AQ, B_AQ, B_LE, omega, S_AQ)
    argument = argument_fn(omega, S_AQ, B_AQ, K_LE, B_LE)
    tide = tide_fn(S_AQ, E_0)
    xi = xi_fn(R_W, R_C, omega, K_AQ, B_AQ, betta)
    h_w = h_w_fn(argument, tide, xi)
    amp = h_w / tide 
    shift = np.angle(h_w / tide, deg=True)
    
    return amp.real, shift.real

# Inversion function to apply least squares
def inv_wang(vars):
    
    K_AQ, S_AQ, K_LE = vars
    
    omega = omega_fn(PERIOD)
    betta = betta_fn(K_LE, K_AQ, B_AQ, B_LE, omega, S_AQ)
    argument = argument_fn(omega, S_AQ, B_AQ, K_LE, B_LE)
    tide = tide_fn(S_AQ, E_0)
    xi = xi_fn(R_W, R_C, omega, K_AQ, B_AQ, betta)
    h_w = h_w_fn(argument, tide, xi)
    
    amp = h_w / tide 
    shift = np.angle(h_w / tide, deg=True)
    
    # Here we build the objective functions. Note that amp_obs and shift_obs are the observed
    # amplitude ratio and shift contained in "amp_ratio" and  "phase_shift" respectively.

    opt_amplitude = float((amp.real - amp_obs*S_AQ)/amp_obs*S_AQ)
    opt_shift = float((shift.real - shift_obs)/shift_obs)
    FO = (opt_amplitude) + (opt_shift)

    return [FO]


In [6]:
# These are the borehole geometry properties

r_w = np.array([156, 203]) * 1E-3 / 2 # radius
r_c = r_w # radius of the case
#b_aq = np.array([12, 28]) # aquifer depth
b_aq = np.array([4, 6]) # aquifer depth
b_le = np.array([25, 28]) # aquitard depth
F = np.ones((5)) * 1.97322 # frequency of the M2 tide
period = 1/F * 24 * 3600 # period of the M2 tide

# Now we choose which borehole we would like to study

borehole = 0 # 0 = B1

amp_obs = amp_ratio.iloc[0][borehole]
shift_obs = phase_shift.iloc[0][borehole]

R_W = r_w[borehole]
R_C = r_c[borehole]
PERIOD = period[borehole]
B_AQ = b_aq[borehole]
B_LE = b_le[borehole]
E_0 = 1E-7 # Signal amplitude [-] (this amplitude is unrealistic, but convinient for this example)


In [7]:
###### ---                                  --- ######

# B1 init conditions
K_init = 1.1E-5 # hydraulic conductivity aquifer
S_init = 1.8E-6 # specific storage at constant strain
KL_init = 5.4E-8# 1E-8 # vertical hydraulic conductivity aquitard

# B2 init conditions
#K_init = 1.5E-5 # hydraulic conductivity aquifer
#S_init = 1.8E-6 # specific storage at constant strain
#KL_init = 1.E-20 # vertical hydraulic conductivity aquitard

In [8]:
# Least squares search

P = least_squares(inv_wang, (K_init, S_init, KL_init), jac = '3-point')#, bounds=([1E-10, 1E-7, 1E-200], [1E-2, 1.8E-6, 1E-6]))

print('Aquifer hydraulic conductivity: ', P.x[0])
print('Specific storage at constant strain: ', P.x[1])
print('Aquitard hydraulic conductivity: ', P.x[2])

print('----------')

found_res = wang_2018(P.x[0], P.x[1], P.x[2])

print('Found ampliturde ratio: ', found_res[0])
print('Real ampliturde ratio: ', amp_obs*P.x[1])

print('Found phase shift: ', found_res[1])
print('Real phase shift: ', shift_obs)

print('----------')

# Compute error of the search, i.e, final value of objetive function

obj_amplitude = float(np.abs(found_res[0] - amp_obs*P.x[1])/amp_obs*P.x[1])
obj_shift = float(np.abs(found_res[1] - shift_obs)/shift_obs)
    
obj_FO = np.abs(obj_amplitude) + np.abs(obj_shift)

print('Goodness of the search: ', obj_FO)


Aquifer hydraulic conductivity:  1.0998860704071588e-05
Specific storage at constant strain:  1.785514435399761e-06
Aquitard hydraulic conductivity:  2.022027908531787e-08
----------
Found ampliturde ratio:  0.6472238288515514
Real ampliturde ratio:  0.15369662380238952
Found phase shift:  33.82973585306424
Real phase shift:  33.82970961411824
----------
Goodness of the search:  7.756286595672479e-07
